# "Tree Cover Loss in X?" Widget

This widget shows the tree cover loss for the selected country and compares to other, similar countries in a ranked list similiar to the '[*Tree Cover Gain in X*](https://goo.gl/D3iFEf)' widget.

Data:

1. adm-0 (adm1 and 2 = None are hardcoded)
2. forest loss
3. extent

User Variables:

1. extent year
2. unit (ha/relative %)
3. locations (all)
4. canopy threshold

## Note for the Front-End

Here I have used a new query `all_loss` to retrieve the total loss and extent of each country as a list...
however, I suspect the same data can be scraped from existing queries (`getLoss` and `extent`, respectively) in the front end by selecting adm0 as `None`.

In [1]:
#Import Global Metadata etc

%run '0.Importable_Globals.ipynb'

In [10]:
# Variables
adm0 = 'BRA'
adm1 = None #Hardcoded
adm2 = None #Hardcoded

extent_year = 2000     # extent data 2000 or 2010
location = "All Region"
thresh = 30

url = f"https://production-api.globalforestwatch.org/v1/query/{ds}"


tags = ["summary", "forest_change"] #replaces current tree cover loss widget in summary

selectable_polynames = ['gadm28', 'primary_forest', 'ifl_2013', 'wdpa', 'primary_forest__wdpa',
                        'mining', 'ifl_2013__wdpa', 'primary_forest__mining', 'ifl_2013__mining',
                        'plantations', 'plantations__mining']

In [114]:
def all_loss(p_name, adm0, extent_year=2000, threshold=30):
    print(f'Request for loss')
    sql = (f"SELECT polyname, year_data.year as year, "
           f"SUM(year_data.area_loss) as loss, "
           f"SUM({extent_year}) as extent, "
           f"FROM data "
           f"WHERE polyname = '{p_name}' "
           f"AND thresh= {threshold} "
           f"GROUP BY polyname, iso")
    return sql


In [115]:
# Query for loss

url = f"https://production-api.globalforestwatch.org/v1/query/{ds}"
loss_sql = all_loss(p_name=polynames[location], adm0=adm0, extent_year=extent_year_dict[extent_year], threshold=thresh)

properties = {"sql": loss_sql}
rl = requests.get(url, params = properties)
print(rl.url)
print(f'Status: {rl.status_code}')

loss_data = rl.json()['data']

#Example data structure
loss_data[0:5]

Request for loss
https://production-api.globalforestwatch.org/v1/query/499682b1-3174-493f-ba1a-368b4636708e?sql=SELECT+polyname%2C+year_data.year+as+year%2C+SUM%28year_data.area_loss%29+as+loss%2C+SUM%28area_extent_2000%29+as+extent%2C+FROM+data+WHERE+polyname+%3D+%27gadm28%27+AND+thresh%3D+30+GROUP+BY+polyname%2C+iso
Status: 200


[{'extent': 519187505.155631,
  'iso': 'BRA',
  'loss': 46369255.51258146,
  'polyname': 'gadm28'},
 {'extent': 279340822.8477372,
  'iso': 'USA',
  'loss': 33754469.852130815,
  'polyname': 'gadm28'},
 {'extent': 7915160.704204235,
  'iso': 'ROU',
  'loss': 298195.74271113425,
  'polyname': 'gadm28'},
 {'extent': 760475347.7894046,
  'iso': 'RUS',
  'loss': 49484662.72782358,
  'polyname': 'gadm28'},
 {'extent': 53185271.23384352,
  'iso': 'MEX',
  'loss': 3099532.0310167223,
  'polyname': 'gadm28'}]

# Ranking

Rank the countries according to loss (ha) or loss relative to extent (%)

i.e. ```relative loss = 100 x (loss / extent)```

In [116]:
"""First clean the data and calculate the loss relative the extent in the user selected year.
Note: if no loss data, set to zero."""

cleaned_data = list(map(lambda x: {
    
    'iso': x.get('iso'),
    'loss': x.get('loss',0),
    'rel_loss': 100*x.get('loss',0)/x.get('extent',1)
    }, loss_data))

cleaned_data[0:5]

[{'iso': 'BRA', 'loss': 46369255.51258146, 'rel_loss': 8.931119307018353},
 {'iso': 'USA', 'loss': 33754469.852130815, 'rel_loss': 12.083615100729357},
 {'iso': 'ROU', 'loss': 298195.74271113425, 'rel_loss': 3.7673997263598693},
 {'iso': 'RUS', 'loss': 49484662.72782358, 'rel_loss': 6.507069936148301},
 {'iso': 'MEX', 'loss': 3099532.0310167223, 'rel_loss': 5.827801492990016}]

In [117]:
#Retrieve the country we're interested in
country_data = list(filter(lambda x: x.get('iso') == adm0, cleaned_data))[0]
country_data

{'iso': 'BRA', 'loss': 46369255.51258146, 'rel_loss': 8.931119307018353}

### Sort by loss (ha)

In [88]:
loss_rank_list = sorted(cleaned_data, key=lambda k: k.get('loss'), reverse=True) 
loss_rank_list[0:3]

[{'iso': 'RUS', 'loss': 49484662.72782358, 'rel_loss': 6.507069936148301},
 {'iso': 'BRA', 'loss': 46369255.51258146, 'rel_loss': 8.931119307018353},
 {'iso': 'CAN', 'loss': 36012126.813233174, 'rel_loss': 8.61576093257477}]

In [92]:
# Get country's rank and print adjacent values ('net_usd' and 'iso' in this case)

rank = 1
for i in loss_rank_list:
    if i.get('iso') == adm0:
        print('RANK =', rank)
        break
    else:
        rank += 1

if rank == 1:
    bottom_bound = -1
    upper_bound = 4
elif rank == 2:
    bottom_bound = 2
    upper_bound = 3    
elif rank == len(loss_rank_list):
    bottom_bound = 5
    upper_bound = -1   
elif rank == len(loss_rank_list)-1:
    bottom_bound = 4
    upper_bound = 0 
else:
    bottom_bound = 3
    upper_bound = 2 
    
loss_rank_list[rank-bottom_bound:rank+upper_bound]

RANK = 2


[{'iso': 'RUS', 'loss': 49484662.72782358, 'rel_loss': 6.507069936148301},
 {'iso': 'BRA', 'loss': 46369255.51258146, 'rel_loss': 8.931119307018353},
 {'iso': 'CAN', 'loss': 36012126.813233174, 'rel_loss': 8.61576093257477},
 {'iso': 'USA', 'loss': 33754469.852130815, 'rel_loss': 12.083615100729357},
 {'iso': 'IDN', 'loss': 23086617.660850815, 'rel_loss': 14.342196752019118}]

### Sort by Relative loss (%)

In [89]:
rel_rank_list = sorted(cleaned_data, key=lambda k: k.get('rel_loss'), reverse=True) 
rel_rank_list[0:3]

[{'iso': 'MRT', 'loss': 33.2423107996583, 'rel_loss': 99.77814699057237},
 {'iso': 'BFA', 'loss': 131.12572610378265, 'rel_loss': 99.25612079050147},
 {'iso': 'UMI', 'loss': 103.86826594173908, 'rel_loss': 71.19232792782252}]

In [91]:
# Get country's rank and print adjacent values ('net_usd' and 'iso' in this case)

rank = 1
for i in rel_rank_list:
    if i.get('iso') == adm0:
        print('RANK =', rank)
        break
    else:
        rank += 1

if rank == 1:
    bottom_bound = -1
    upper_bound = 4
elif rank == 2:
    bottom_bound = 2
    upper_bound = 3    
elif rank == len(rel_rank_list):
    bottom_bound = 5
    upper_bound = -1   
elif rank == len(rel_rank_list)-1:
    bottom_bound = 4
    upper_bound = 0 
else:
    bottom_bound = 3
    upper_bound = 2 
    
rel_rank_list[rank-bottom_bound:rank+upper_bound]

RANK = 49


[{'iso': 'AUS', 'loss': 3961396.713502459, 'rel_loss': 9.36280216154266},
 {'iso': 'ABW', 'loss': 2.254775032401085, 'rel_loss': 9.036301035276807},
 {'iso': 'BRA', 'loss': 46369255.51258146, 'rel_loss': 8.931119307018353},
 {'iso': 'NZL', 'loss': 1004101.5595766604, 'rel_loss': 8.899452185499824},
 {'iso': 'ATF', 'loss': 6.33406925201416, 'rel_loss': 8.842007289642128}]

# Dynamic Sentence

In [109]:
if location == 'All Region':
    location_phrase = 'region-wide'
else:
    location_phrase = 'in ' + location.lower()

print(f"Between 2001 and 2016, {iso_to_countries[adm0]} lost ",end="")
print(f"{country_data.get('loss')}ha of tree cover {location_phrase}, ",end="")
print(f"equivalent to a {country_data.get('rel_loss')}% loss ",end="")
print(f"relative to {extent_year} tree cover extent.",end="")

Between 2001 and 2016, Brazil lost 46369255.51258146ha of tree cover region-wide, equivalent to a 8.931119307018353% loss relative to 2000 tree cover extent.